Imports

In [111]:
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectFromModel
import pickle
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

Import dataset

In [96]:
with open('shape_features.csv', 'r') as data:
    df = pd.read_csv(data)

Shuffle data

In [97]:
from sklearn.utils import shuffle
data = shuffle(df)

Separate features from labels, and split into train and test sets. Drop useless features


In [768]:
from sklearn.model_selection import train_test_split

X = df.iloc[:,0:-1]
y = df.iloc[:,-1]
columns = X.columns
print(columns)
X = X.drop(columns=['Max speed squared', 'Min speed squared', 'Total duration', 'Bounding box area', 'Movement', 'Bounding box height', 'Bounding box width', 'Length ratio'])

Index(['Angle of BB diagonal', 'Aspect', 'Average curvature',
       'Average squared speed', 'Bounding box area', 'Bounding box diagonal',
       'Bounding box height', 'Bounding box width', 'Convex hull area ratio',
       'Cos of first to last', 'Cos of initial angle', 'Curviness', 'DCR',
       'Density 1', 'Density 2', 'Distance first to last',
       'Enclosing shape5 ratio', 'Entropy', 'Least shape3 error',
       'Length perimeter ratio', 'Length ratio', 'Log BB area', 'Log aspect',
       'Log longest BB side', 'Log total length', 'Max curvature',
       'Max speed squared', 'Min speed squared', 'Movement', 'NDDE',
       'Openness', 'Overtracing', 'Perimeter efficiency', 'Perimeter to area',
       'Point ratio', 'Sharpness', 'Sin of first to last',
       'Sin of initial angle', 'Thinness ratio', 'Total angle',
       'Total angle / total length', 'Total duration', 'Total length',
       'Total length / BB diagonal', 'Width to height ratio'],
      dtype='object')


Feature Selection by calculating mutual information between features, and selecting the top 25



In [769]:
from sklearn.feature_selection import mutual_info_classif, f_classif
X = SelectKBest(score_func=mutual_info_classif,k=25).fit_transform(X, y)
X = pd.DataFrame(X)

Split into training and test sets

In [777]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)

Dataset Statistics

In [771]:
print(str(len(X_train)) + " samples in the training set")
print(str(len(X_test)) + " samples in the test set")
print(str(len(X_train.columns)) + " features")
print(str(len(set(y_train))) + " labels")
print("Labels are:", set(y_train))
print("Features are:", set(X_train.columns))

189 samples in the training set
21 samples in the test set
25 features
5 labels
Labels are: {'shape3', 'shape2', 'shape4', 'shape5', 'shape1'}
Features are: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24}


Train neural net

In [826]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(activation='relu', hidden_layer_sizes=(20), solver='adam', verbose=False, random_state=3)
model.fit(X_train,y_train)
model.score(X_test, y_test)


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


0.8571428571428571

Print Classification Report

In [829]:
from sklearn.metrics import classification_report
print(classification_report(model.predict(X_test),y_test))

              precision    recall  f1-score   support

      shape1       1.00      0.75      0.86         4
      shape2       1.00      1.00      1.00         6
      shape3       0.62      1.00      0.77         5
      shape4       1.00      1.00      1.00         4
      shape5       0.00      0.00      0.00         2

    accuracy                           0.86        21
   macro avg       0.72      0.75      0.73        21
weighted avg       0.82      0.86      0.82        21



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Save Model

In [827]:
pickle.dump(model, open('Ex5_shape_classifier.sav','wb'))